# 最近話題の[JAX](https://jax.readthedocs.io/en/latest/installation.html) の勉強をしているのでこのサイトたち[クイックスタート](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)と関数について詳しく説明していてくれる[サイト](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html)を参考に説明していきます．

まず, こちらの[サイト]((https://jax.readthedocs.io/en/latest/notebooks/quickstart.html))を参考に説明していきます．

JAXはXLAをnumpyのコードを走らせるときやコンパイルする時により早くするために使っています，

例えば，乗算, 和算, 減算がある関数内であったときに, XLAなしでこれを使うとそれぞれの計算に一つずつのGPUカーネルを用意し, 計算しますがXLAありで使うとカーネルを一つだけのGPUカーネルで済むことを述べています．

In [1]:
import jax.numpy as jnp #numpyと同じようなもの
from jax import grad #微分する時とか勾配計算したいとき
from jax import jit #コードを早くしたいとき
from jax import vmap #ベクトルに変換したいとき
from jax import random 

----

numpyではnumpy.random.seed(0)のようなランダムに決めた値でも再現性を持たせるための関数を持ち合わせています．

一方, jaxでもそのようなkeyを持ち合わせています．

In [2]:
key = random.PRNGKey(0)
x = random.uniform(key,(10,)) #一様分布からランダムな値を取得
print(x)
print(x)
print(x)

[0.35490513 0.60419905 0.4275843  0.23061597 0.32985854 0.43953657
 0.25099766 0.27730572 0.7678207  0.71474564]
[0.35490513 0.60419905 0.4275843  0.23061597 0.32985854 0.43953657
 0.25099766 0.27730572 0.7678207  0.71474564]
[0.35490513 0.60419905 0.4275843  0.23061597 0.32985854 0.43953657
 0.25099766 0.27730572 0.7678207  0.71474564]


上の結果を見ればわかるようにしっかりとランダムだが，再現性を持つ関数になっていることがわかりますね．

---

次に上記で説明したコードを早くするjit(デコレータ)を見ていきましょう.


In [9]:
def func(x,a = 2,b = 3):
    return jnp.exp(a) + b * jnp.sin(b)

x = random.normal(key,(100000,))
%timeit func(x).block_until_ready()

27.1 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
func_jit = jit(func)
%timeit func_jit(x).block_until_ready()

4.7 µs ± 87.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


一回ごとのループは早くなっていることがわかりますね．

しかしながら, jitを適応できない場合もあります．

あとで説明するのですが，簡潔に説明すると, 関数内で使う行列の型が引数などになっているときです．

----

次はgradの説明をしていきます．

関数の傾きなどを計算したいときにこの関数は使われます．

早速例を見ていきます．

In [15]:
def func_1(x):
    return x ** 2 + 3
x = 1.0
derivative_func = grad(func_1)
print(derivative_func(x))

2.0


func_1は $x ^ 2 + 3$ なので微分は $2x$ なので $x = 1$ の傾きは2.0であってますね．

機械学習の世界では行列にたいての傾きなどを求めたい時が多々あるのでその例を見てみましょう．

なので,次はxにベクトルを入れてみましょう．

In [16]:
def func_1(x):
    return x ** 2 + 3
x = jnp.arange(3.)
derivative_func = grad(func_1)
print(derivative_func(x))


TypeError: Gradient only defined for scalar-output functions. Output had shape: (3,).

エラーが出てしまいますね．

メッセージを見てみると, 勾配はスカラー値じゃないとできないよと言っています．

そこで1番最初に紹介したvmapを適応してみます．

In [21]:
def func_1(x):
    return x ** 2 + 3
x = jnp.arange(3.)
derivative_func = vmap(grad(func_1))
print(derivative_func(x))

[0. 2. 4.]


うまくいきましたね．
このような使い方もvmapはできます．

vmapの他の例も見てみましょう．

In [28]:
mat = random.normal(key, (150,100))
batched_x = random.normal(key, (10,100))

def apply_matrix(v):
    return mat @ v

print(mat.shape,batched_x.shape)

(150, 100) (10, 100)
(10, 100)


In [31]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched]) # for loopでvの型は100なので行列計算できる．

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
1.68 ms ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


上の計算は書くのが面倒な上に遅いので，vmapを使うと次のように書き換えられます．

In [34]:
@jit
def vmap_batch_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batch_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
19.6 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


---



ここからはこちらの[サイト](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html)を参考に勉強していきます．

このサイトでJAXについては以下の説明がしてあります．

*JAXは,数値プログラムの変換を表現・構成するための言語です.JAXは,CPUやアクセラレータ（GPU/TPU）用の数値計算プログラムをコンパイルすることもできます.JAXは,数値プログラムの変換を表現・構成するための言語であり,多くの数値プログラムや科学プログラムに最適ですが,以下に説明する特定の制約条件下で書かれたものに限ります.*

次の例は面白い例のですが，関数を一度呼び出すと，以降に同じデータ型を入力する前にグローバル変数を変えてもそれが反映しないという例です．

In [37]:
g = 0.
def impure_uses_globals(x):
  return x + g


print (jit(impure_uses_globals)(4.))
g = 10.  
print ("global変数を変えたとき", jit(impure_uses_globals)(5.))

print ("入力データ型を変えた", jit(impure_uses_globals)(jnp.array([4.])))

4.0
global変数を変えたとき 5.0
入力データ型を変えた [14.]


---

JAXは関数型プログラミングのため，jitしたい関数の中でイテレータを使うのはよくないとされています．

実際に例を見ていきましょう．

JAXにはfori_loopというものがあり, 簡潔に説明するとfor_loopみたいな感じです．

引数には(ループを始める数値, ループを終了する数値, 回したい関数, 回したい関数に入れる初期値)

In [42]:
import jax.numpy as jnp
import jax.lax as lax

array = jnp.arange(10)
print(lax.fori_loop(0,10,lambda i,x : x + array[i],0))
print(array.sum())

45
45


上を見てみるとイテレータを使わないと値が一致していることがわかりますね．

In [43]:
iterator = iter(range(10))
print(lax.fori_loop(0,10,lambda i,x: x + next(iterator),0))

0


上からイテレータを使うとうまく計算できていませんね．

---

JAXでは行列の値を変えるときには次のような操作をして変えることができます．

numpyのような変換の仕方をするとエラーが出てしまうので注意です．

In [45]:
array = jnp.zeros((3,3))

array[1,1] = 1 #numpyのように書くとエラーが出る

TypeError: '<class 'jaxlib.xla_extension.Array'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html

In [46]:
array = array.at[1,1].set(1.0)
print(array)

[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


上のように更新することで行列の(1,1)部分が書き換えれることがわかりましたね．

---

次に, 1番最初に説明したjitについてのエラー例を見てみましょう．

In [47]:
@jit
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * 4
    
print(f(3))

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
The error occurred while tracing the function f at /var/folders/rn/8ylp503d60g0xr_qm3ghjknr0000gn/T/ipykernel_2472/352147184.py:1 for jit. This concrete value was not available in Python because it depends on the value of the argument 'x'.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError

このように引数が原因でリターン値が変わるのはよくないとされています．

この理由はJAXの関数は値によって条件分岐しない仮定でコンパイルの時間を節約するために使いまわしているからです．

解決策としては下の例に挙げます．

In [48]:
from functools import partial

@partial(jit,static_argnames=('x'))
def f(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * 4
    
print(f(4))

-16
